In [1]:
!pip install streamlit
! pip install pdf2image
!apt-get install poppler-utils 


     |████████████████████████████████| 9.1 MB 4.7 MB/s 
     |████████████████████████████████| 111 kB 39.7 MB/s 
     |████████████████████████████████| 4.3 MB 28.5 MB/s 
     |████████████████████████████████| 178 kB 55.0 MB/s 
     |████████████████████████████████| 76 kB 4.1 MB/s 
     |████████████████████████████████| 180 kB 56.7 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 125 kB 56.7 MB/s 
     |████████████████████████████████| 790 kB 37.2 MB/s 
     |████████████████████████████████| 374 kB 49.2 MB/s 
  Created wheel for pympler: filename=Pympler-0.9-py3-none-any.whl size=164824 sha256=1f3d7559e89a3b2bb039a139dd0a4d554c22fad84849786dba73299812ac16b2
  Stored in directory: /root/.cache/pip/wheels/1a/f3/d8/35d5614ea4ddd295ffb9372a5f2f9570d9593d1ea4be33ec6d
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=ccfa57b7b4882fb5e313ec237d33bae688331b4556557281b50992393b09f37b
  Stored in direc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 1s (224 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [2]:
!pip install albumentations==0.4.6
!sudo apt install tesseract-ocr
!pip install pytesseract

     |████████████████████████████████| 117 kB 5.2 MB/s 
     |████████████████████████████████| 948 kB 23.2 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65172 sha256=faa1674916be7f9a1dd9833ff7c732e863d3ee47d897d1f490f1216a1b511401
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tess

In [67]:
%%writefile app.py
import streamlit as st
import pdf2image
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import time
from datetime import datetime
import torch
import torch.nn as nn
import torchvision
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytesseract
from io import StringIO


pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

TRANSFORM = A.Compose([
                #ToTensor --> Normalize(mean, std)
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value = 255,
                ),
                ToTensorV2()
            ])

class DenseNet(nn.Module):
    def __init__(self, pretrained = True, requires_grad = True):
        super(DenseNet, self).__init__()
        denseNet = torchvision.models.densenet121(pretrained=True).features
        self.densenet_out_1 = torch.nn.Sequential()
        self.densenet_out_2 = torch.nn.Sequential()
        self.densenet_out_3 = torch.nn.Sequential()

        for x in range(8):
            self.densenet_out_1.add_module(str(x), denseNet[x])
        for x in range(8,10):
            self.densenet_out_2.add_module(str(x), denseNet[x])

        self.densenet_out_3.add_module(str(10), denseNet[10])

        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, x):

        out_1 = self.densenet_out_1(x) #torch.Size([1, 256, 64, 64])
        out_2 = self.densenet_out_2(out_1) #torch.Size([1, 512, 32, 32])
        out_3 = self.densenet_out_3(out_2) #torch.Size([1, 1024, 32, 32])
        return out_1, out_2, out_3

class TableDecoder(nn.Module):
    def __init__(self, channels, kernels, strides):
        super(TableDecoder, self).__init__()
        self.conv_7_table = nn.Conv2d(
                        in_channels = 256,
                        out_channels = 256,
                        kernel_size = kernels[0],
                        stride = strides[0])
        self.upsample_1_table = nn.ConvTranspose2d(
                        in_channels = 256,
                        out_channels=128,
                        kernel_size = kernels[1],
                        stride = strides[1])
        self.upsample_2_table = nn.ConvTranspose2d(
                        in_channels = 128 + channels[0],
                        out_channels = 256,
                        kernel_size = kernels[2],
                        stride = strides[2])
        self.upsample_3_table = nn.ConvTranspose2d(
                        in_channels = 256 + channels[1],
                        out_channels = 1,
                        kernel_size = kernels[3],
                        stride = strides[3])

    def forward(self, x, pool_3_out, pool_4_out):
        x = self.conv_7_table(x)  #[1, 256, 32, 32]
        out = self.upsample_1_table(x) #[1, 128, 64, 64]
        out = torch.cat((out, pool_4_out), dim=1) #[1, 640, 64, 64]
        out = self.upsample_2_table(out) #[1, 256, 128, 128]
        out = torch.cat((out, pool_3_out), dim=1) #[1, 512, 128, 128]
        out = self.upsample_3_table(out) #[1, 3, 1024, 1024]
        return out

class ColumnDecoder(nn.Module):
    def __init__(self, channels, kernels, strides):
        super(ColumnDecoder, self).__init__()
        self.conv_8_column = nn.Sequential(
                        nn.Conv2d(in_channels = 256,out_channels = 256,kernel_size = kernels[0], stride = strides[0]),
                        nn.ReLU(inplace=True),
                        nn.Dropout(0.8),
                        nn.Conv2d(in_channels = 256,out_channels = 256,kernel_size = kernels[0], stride = strides[0])
                        )
        self.upsample_1_column = nn.ConvTranspose2d(
                        in_channels = 256,
                        out_channels=128,
                        kernel_size = kernels[1],
                        stride = strides[1])
        self.upsample_2_column = nn.ConvTranspose2d(
                        in_channels = 128 + channels[0],
                        out_channels = 256,
                        kernel_size = kernels[2],
                        stride = strides[2])
        self.upsample_3_column = nn.ConvTranspose2d(
                        in_channels = 256 + channels[1],
                        out_channels = 1,
                        kernel_size = kernels[3],
                        stride = strides[3])

    def forward(self, x, pool_3_out, pool_4_out):
        x = self.conv_8_column(x)  #[1, 256, 32, 32]
        out = self.upsample_1_column(x) #[1, 128, 64, 64]
        out = torch.cat((out, pool_4_out), dim=1) #[1, 640, 64, 64]
        out = self.upsample_2_column(out) #[1, 256, 128, 128]
        out = torch.cat((out, pool_3_out), dim=1) #[1, 512, 128, 128]
        out = self.upsample_3_column(out) #[1, 3, 1024, 1024]
        return out

class TableNet(nn.Module):
    def __init__(self):
        super(TableNet, self).__init__()

        self.base_model = DenseNet(pretrained = False, requires_grad = True)
        self.pool_channels = [512, 256]
        self.in_channels = 1024
        self.kernels = [(1,1), (1,1), (2,2),(16,16)]
        self.strides = [(1,1), (1,1), (2,2),(16,16)]

        #common layer
        self.conv6 = nn.Sequential(
            nn.Conv2d(in_channels = self.in_channels, out_channels = 256, kernel_size=(1,1)),
            nn.ReLU(inplace=True),
            nn.Dropout(0.8),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size=(1,1)),
            nn.ReLU(inplace=True),
            nn.Dropout(0.8))

        self.table_decoder = TableDecoder(self.pool_channels, self.kernels, self.strides)
        self.column_decoder = ColumnDecoder(self.pool_channels, self.kernels, self.strides)

    def forward(self, x):

        pool_3_out, pool_4_out, pool_5_out = self.base_model(x)
        conv_out = self.conv6(pool_5_out) #[1, 256, 32, 32]
        table_out = self.table_decoder(conv_out, pool_3_out, pool_4_out) #torch.Size([1, 1, 1024, 1024])
        column_out = self.column_decoder(conv_out, pool_3_out, pool_4_out) #torch.Size([1, 1, 1024, 1024])
        return table_out,column_out

@st.cache(allow_output_mutation=True)
def load_model():

    model = TableNet()
    model.load_state_dict(torch.load("densenet_config_4_model_checkpoint.pth.tar",map_location ='cpu')['state_dict'])
    model.eval()
    return model

def predict(img_path):
    with st.spinner('Processing..'):
        orig_image = img_path[0].resize((1024, 1024))
        test_img = np.array(orig_image.convert('LA').convert("RGB"))

        now = datetime.now()
        image = TRANSFORM(image = test_img)["image"]
        with torch.no_grad():
            image = image.unsqueeze(0)
            #with torch.cuda.amp.autocast():
            table_out, _  = model(image)
            table_out = torch.sigmoid(table_out)

        #remove gradients
        table_out = (table_out.detach().numpy().squeeze(0).transpose(1,2,0) > 0.5).astype(np.uint8)

        #get contours of the mask to get number of tables
        contours, table_heirarchy = cv2.findContours(table_out, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        table_contours = []
        #ref: https://www.pyimagesearch.com/2015/02/09/removing-contours-image-using-python-opencv/
        #remove bad contours
        for c in contours:

            if cv2.contourArea(c) > 3000:
                table_contours.append(c)

        if len(table_contours) == 0:
            st.write("No Table detected")

        table_boundRect = [None]*len(table_contours)
        for i, c in enumerate(table_contours):
            polygon = cv2.approxPolyDP(c, 3, True)
            table_boundRect[i] = cv2.boundingRect(polygon)

        #table bounding Box
        table_boundRect.sort()

        orig_image = np.array(orig_image)
        #draw bounding boxes
        color = (0,0,255)
        thickness = 4

        for x,y,w,h in table_boundRect:
            cv2.rectangle(orig_image, (x,y),(x+w,y+h), color, thickness)

        st.image(orig_image)

        end_time = datetime.now()
        difference = end_time - now
        #print("Total Time : {} seconds".format(difference))
        time = "{}".format(difference)

        st.write(f"{time} secs")

        st.write("Predicted Tables")

        image = test_img[...,0].reshape(1024, 1024).astype(np.uint8)

        for i,(x,y,w,h) in enumerate(table_boundRect):
            image_crop = image[y:y+h,x:x+w]
            st.image(image_crop)
            data = pytesseract.image_to_string(image_crop)
            try:
                df = pd.read_csv(StringIO(data),sep=r'\|',lineterminator=r'\n',engine='python')
                st.write(f" ## Table {i+1}")
                st.write(df)
            except pd.errors.ParserError:
                try:
                    df = pd.read_csv(StringIO(data),delim_whitespace=True,lineterminator=r'\n',engine='python')
                    st.write(f" ## Table {i+1}")
                    st.write(df)
                except pd.errors.ParserError:
                    st.write(f" ## Table {i+1}")
                    st.write(data)



with st.spinner("Loading Last Checkpoint"):
    model = load_model()

st.header("Data Extraction from Tables")
#upload files
file = st.file_uploader("Please upload an Image file")


if file is not None:
    images = pdf2image.convert_from_bytes(file.read())
    pic=images
    st.write(type(pic))
    st.image(pic, use_column_width=True)
    predict(pic)
    
  

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

npx: installed 22 in 3.46s
your url is: https://tender-bat-76.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.184.19.124:8501

